# 自然语言处理
----------------

## $\S1 $. 词汇表征
### 1.1 词汇表示(word representation)
#### 一. one-hot与featurized representation  
之前, 我们一直用词汇表来表征词汇,即用单词在Directionary中的位置做representation的方法, 将其称为one-hot. 

1. one-hot词汇编码
  1. 这种表示方法, 把每个单词看做完全独立的, 没有逻辑关系的. 
      因而若用one-hot进行语言学习, 即使已经学习到了"I want apple $\underline { jucie } $" , 也不会知道"I want orange "$\underline { \quad   } $"的横线上应该填什么, 即使我们知道apple和orange所表示的意思极为相似  
  2. 而且one-hot由于用一个向量表示整个词典, 造成单词表示的向量非常大      
2. 特征化表示单词 - featurized representation
  1. 特征化表示,给单词在不同属性上打分, 这些分数代表该单词的意思和该feature的相关程度. 如下, 我们从Gender,Royal,Age,Food4个方面来标注单词.<img src="../img/wordrepresentation.png" width="60%" height="60%">
  2. 此方法下, 我们要表示5391号单词Men时, 使用向量${ e }_{ 5394 }={ [-1,0.01,0.03,0.09] }^{ T }$表示
  3. 这种向量化的表现方式就可以给出词之间的相似度, 从而判断出apple和orange很相似
  4. 词表示可视化: 当我们用向量化表示但此后,一种算法将高维向量展现成二维向量: [T-sne](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html)  
  T-sne展现的图像, 使得意义相近的词汇距离的很近<img src="../img/t-sne.png" width="30%" height="30%">
3. 词嵌入: word-embeding  
 若我们此时用300个feature表示这些单词, 则每个单词变成了300维向量.就好像每个单词都嵌入在一个300维空间内的某个点.因此featurized representation也称为词嵌入. 
 
4. 词嵌入矩阵
  1. 若我们用300个feature表示每个单词, 单词表一共有10000个单词, 则最终得到10000个规模为$R^{300}$的向量组成的矩阵.这个矩阵就是次潜入矩阵$R^{3000*10000}$  
  2. 我们从嵌入矩阵 $E$ 中获取某个单词的向量:
  假设获取词典中第2624号单词的词嵌入,其one-hot编码为$o_{2624}={ [0,0,..,1,..0] }^{ T }$( 1在2624位置,其余位置为0 ),则其词嵌入$e_{2624}=E*o_{2624}$
  
  
### 1.2 词嵌入 (word embeding)
#### 一. NLP中的词嵌入
1. 从识别人名开始
  1. "Sally Johnson is an orange farmer" 与 "Robert Lin is a durian culture"两句子中, 找出哪个是人名.NLP对这个问题的解决思路是, 当NLP发现句子尾部的"orange farmer"和"durian culture"后, 就会猜测句首的"Sally Johnson"和"Robert Lin"可能是人名
  2. NLP要获取"orange"和"durian"都是水果的知识, 需要从超大量文本中学习, 可能是100亿个句子.从100个句子中学习到每个单词的feature就是词嵌入.而通过迁移学习, 我们就能把这些已经获得的关于单词的知识(词嵌入), 应用到我们需要的更少量的词汇表中
  
2. 应用迁移学习和词嵌入的步骤  
  1. 从大量文本中学习词嵌入(1-100B单词训练量),或者下载已经训练好的词嵌入向量
  2. 迁移词向量应用到新任务中(更小规模的数据集,100k单词量)
  3. 可选步骤: 使用新数据集更精细化调整词嵌入

#### 二. 类比推论 - analogy reasoning
1. 词嵌入的一个小规模应用就是发现"词对"之间的相似性, 如词对"men,women"相似于词对"king,queen"
2. 因为词嵌入把每个单词表示为向量, 所以词对之间的举例可以用来表示2个单词之间的关系.  
  比如:${ e }_{ man }-{ e }_{ women }={ [2,0,0,0] }^{ T }$, 表示"man"和"women"在gender属性上差别较大. 同时我们发现,${ e }_{ king }-{ e }_{ queen }$的结果也是${ [2,0,0,0] }^{ T }$.   
  因此(man,women)类比于(king,queen)
  
#### 三. 学习词嵌入
1. 预测"I want a glass of orange $\underline { \quad  } $"划线处单词.  
 用$O_{i}$表示单词的one-hot编码, $E$ 表示学习到的词嵌入矩阵, $e_{i}$表示单词的词向量 
 <img src="../img/softmaxnlp.png" width="50%" heoght="50%">
  1. 我们先得到每个单词的词向量表示, 预测划线部分单词时, 发现前面共6个单词作, 于是把6设置为history window的大小, 在语句库的所有句子上训练以6个单词为输入的神经网络, 最终softmax输出一个$R^{10000}$的向量, 给出划线部分是单词表中每个单词的概率.  
  2. 若每个单词的词嵌入为$R^{300}$, 则6个单词共1800个数字作为输入.语句库的所有句子训练出的参数$w^{[1]},b^{[1]}$和$w^{[2]},b^{[2]}$即为神经网络的参数.最终再把"I want a glass of orange"的词嵌入作为输入,预测划线部分单词 
2. 在上个句子预测中, 划线部分的单词为"target object", 而前面的6个用于预测的单词为"context". context的不同选择方式决定了算法的不同

## $\S2 $. word2vec模型
简单且高效的学习词嵌入的方法: 分为skip-grams和CBOW
### 2.1 Skip-grams模型
#### 一. Skip-grams与监督学习
1. Skip-grams是一个选取context和target, 构造一个监督学习问题的解决办法. 
通常固定一个单词作为context, 然后选择正负n个词距中随机的一个单词作为target. 而这监督学习问题, 不是要解决问题本身, 而是用来构造更好地词嵌入模型.  
 eg: 固定`context`为"orange", 而选择词距内的另一个单词作为target  
  <img src="../img/skipgrams1.png" width="50%" height="50%">  
  
#### 二. skip-grams训练词嵌入
1. 假设文本集中有一句话为"I want an orange juice". 词典中有10,000k个单词.   
 我们把初始的词嵌入矩阵设置为单位矩阵$E$ , 用单词的初始词向量向量(${ e }_{ c }=E*{ o }_{ c }$), 通过softmax训练给定的target.  
 在softmax反向传播时, 把反向传播一直作用到input的词向量上, 最终得到更精细化(refine)的词向量.  
$\quad \quad \quad \quad\quad \quad \quad x\quad \quad \quad \longrightarrow \quad \quad  y\\ context\quad c\quad ("orange")\quad \longrightarrow \quad target\quad t\quad ("juice")\\ { o }_{ c }\rightarrow E\rightarrow { e }_{ c }({ e }_{ c }=E*{ o }_{ c })\quad \longrightarrow \quad softmax单元\quad \longrightarrow \quad \widehat { y } $  
  1. softmax最终给出,在context条件下得到的target单词,和已知标签y相符概率: $p(t|c)=\frac { exp({ { \theta  }_{ t } }^{ T }{ e }_{ t }) }{ \sum _{ j=1 }^{ 10,000 }{ exp({ { \theta  }_{ j } }^{ T }{ e }_{ j }) }  } $  
  2. softmax损失函数: $L(\widehat { y } ,y)=-\sum _{ i=1 }^{ 10,000 }{ { y }_{ i }\log { { (y }_{ i }) }  } $
  
  
#### 三. skip-grams存在的问题
1. 超大计算量:   
 skip-gram最后一步,用softmax计算划线部分为词汇表每个单词的概率, 这个操作需要超大的计算量.$p(t|c)=exp({ { \theta  }_{ t } }^{ T }{ e }_{ t })/\sum _{ j=1 }^{ 10,000 }{ exp({ { \theta  }_{ j } }^{ T }{ e }_{ j }) } $. 分母部分计算10,000个和速度很慢.而多数情况下, 单词表中不止有10,000个单词, 而会有100,000甚至更大的单词量.为减小计算量产生了几个解决办法 :  
  1. 使用分层的softmax分类器. 不直接给出单词表每个单词的概率, 而是简介给出在单词表中位置的概率,>5k位置还是小于5k位置  
  <img src="../img/hireachysoftmax2.png" width="70%" height="70%">
  2. 实际应用中, 分层softmax分类器不是一位置划分, 而是在第一层放入常见的单词(apple,orange等), 下面放入不常见的单词(durian等)
2. 如何对上下文context取样  
 一旦对context取样完毕, target就会在context的正负10个词距内. 那如何对context取样呢.如果在语料库中均匀且随机采样单词作为context,将会频繁得到很多像"the,a,an,of.."类的单词, 而像"orange,durian,apple"这样的单词却很少出现.这样做会使得我们花很大的力气更新那些不常见的单词的词嵌入向量. 因此, 我们需要一种启发平衡的方式进行采样

### 2.2 负采样 Negtive-sampling
#### 1. 负采样模型的改进
1. skip-grams的监督学习模型, 由于要计算softmax函数, 其分母$\sum _{ j=1 }^{ 10,000 }{ exp({ { \theta  }_{ j } }^{ T }{ e }_{ j }) } $的累加和操作引起大量计算. 因此产生了改进算法"负采样"
2. 负采样( Negtive-sampling)仍然是构造一个监督学习问题, 训练集的采样方式如下 :   
  1. 从文本集的语句中采样一个正例:词对(orange,juice),标签1(代表这个词对确实出现在语句中).
  2. 固定context:orange,随机从词典中选择一个单词作为target.构造负例,标签0(代表这个词对从未出现在语句中).需要注意的是,即使词典中随机采样的单词确实在文本集中出现过, 也要认为它是负例  
  
#### 2. 负采样的一个例子  
1. 文本集中的一个句子: "I want an orange juice". 负采样可构造如下的训练集 (1:4)   
 $(context,target)\quad \quad \quad \quad \quad label\\ orange,juice\quad \quad \quad \quad \quad \qquad 1\quad 正例\\ orange,king\quad \quad \quad \quad \quad \quad \quad 0\quad 负例\\ orange,the\quad \quad \quad \quad \quad \quad \quad \quad 0\quad 负例\\ orange,of\quad \quad \quad \quad \quad \quad \qquad 0\quad 负例\\ orange,book\quad \quad \quad \quad \quad \quad \quad 0\quad 负例$   
2. 负例与正例的采样比k的选择:  
  1. 当文本集的语句很多时, 可以选择k为 $2\sim 5$
  2. 当文本集的语句很少时,选择k为 $5\sim  20$
  
#### 3. 如何训练负采样生成的样本集
训练6257号单词orange的词向量过程 :  
1. 得到orange的初始词向量 : ${ e }_{ 6257 }={ o }_{ 6257 }*{ E }^{ 300*300 }$, 词向量e=(one hot)*词嵌入矩阵(300个feature)
2. 用10,000个logistic分类器训练orange与这10,000个单词是否构成( context,target )组合. 而每个分类器都由负采样,共k+1个样本组成  
<img src="../img/negticesample.png" width="30%" height="30%">
3. 每个logistic分类器: $p(y=1|c,t)=sigmiod({ { \theta  }_{ t } }^{ T }{ e }_{ t })$, 对比skip-grams的$p(t|c)=exp({ { \theta  }_{ t } }^{ T }{ e }_{ t })/\sum _{ j=1 }^{ 10,000 }{ exp({ { \theta  }_{ j } }^{ T }{ e }_{ j }) } $, 已经大大减小计算量

### 2.3 更简单的GloVe词向量法